In [2]:
from openai import OpenAI
import json

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
len(model.encode('My name is Alex.'))

768

In [13]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
operations = []

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [15]:
operations[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.04103037342429161,
  0.02583414688706398,
  -0.03680191561579704,
  -0.020898308604955673,
  -0.020596222952008247,
  0.009353749454021454,
  -0.003331621875986457,
  -0.009491918608546257,
  0.030117960646748543,
  0.019082099199295044,
  0.012690030038356781,
  -0.017078785225749016,
  -0.0016323721501976252,
  0.12997256219387054,
  0.030969271436333656,
  -0.025823596864938736,
  0.027823062613606453,
  0.025159751996397972,
  -0.08081215620040894,
  -0.0036174245178699493,
  -0.008902010507881641,
  0.0034048687666654587,
  -0.023009341210126877,
  -0.034045297652482986,
  0.024598613381385803,
  0.013545605354011059,
  -0.025438982993364334,
  0.011951032094657421,
  -0.020540105178952217,
  -0.010077491402626038,
  0.02057531848549

In [16]:
from elasticsearch import Elasticsearch

In [17]:
es_client = Elasticsearch('http://localhost:9200')

In [18]:
es_client.info()

ObjectApiResponse({'name': 'a532f80da7e9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Hcg_9gQkQH26n0YCib7wUA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [19]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type" : "dense_vector", "dims":768, "index" : True, "similarity" : "cosine"}
        }
    }
}

In [21]:
index_name = 'course-questions'

In [22]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [27]:
q = 'windows or mac?'
vector_search = model.encode(q)

In [28]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search,
    "k" : 3,
    "num_candidates" : 10000
}

In [32]:
res = es_client.search(index = index_name, knn = query, source = ["text", "section","question", "course"])

In [35]:
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'T3RLsZABBYoPZE1FWHG-',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'YnRLsZABBYoPZE1FZHQT',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a